# Mini-projet - Récupérer des méta-données EXIF d'une image

Pour commencer on charge le module `pillow` ...

In [1]:
import sys
!{sys.executable} -m pip install pillow

... puis on l'importe.

In [2]:
# pour travailler sur des images numériques
from PIL import Image

## Récupération des méta-données

In [3]:
# on charge l'image
photo0 = Image.open("chouchou0.jpg")
# on récupère les méta-données
meta0 = photo0._getexif()

1. Ces méta-données sont stockées sous la forme d'un dictionnaire.

In [4]:
# On peut lire ces données avec la boucle POUR sur les clefs et valeurs.
for clef, valeur in meta0.items() :
    print(clef, " : ", valeur)

34853  :  {1: 'N', 2: (47.0, 27.0, 59.0), 3: 'W', 4: (0.0, 33.0, 5.0), 6: 0.0}
296  :  2
34665  :  212
271  :  SAMSUNG
272  :  SM-G531F
305  :  GIMP 2.10.8
274  :  1
306  :  2019:09:29 19:26:26
531  :  1
282  :  72.0
283  :  72.0
36864  :  b'0220'
37121  :  b'\x01\x02\x03\x00'
37377  :  8.724502563476562
36867  :  2019:09:19 16:26:26
36868  :  2019:09:19 16:26:26
37378  :  2.274993896484375
37379  :  6.0
37380  :  0.0
37381  :  2.274993896484375
37383  :  2
37384  :  0
37385  :  24
37386  :  3.3000030517578125
40961  :  65535
40962  :  3264
41990  :  0
40963  :  1836
41992  :  0
41993  :  0
41994  :  0
41996  :  0
41495  :  2
33434  :  0.002364066193853428
33437  :  2.2
41729  :  b'\x01'
34850  :  2
34855  :  50
41986  :  0
40960  :  b'0100'
41987  :  0
41988  :  1.0
41989  :  31
37500  :  b'\x07\x00\x02\x00\x07\x00\x01\x00\x00\x00\x00\x00\x00\x00\x0c\x00\x07\x00\x01\x00\x00\x00\x00\x00\x00\x00@\x00\x07\x00\x01\x00\x00\x00\x00\x00\x00\x00P\x00\x07\x00\x01\x00\x00\x00\x01\x00\x00\x00`\x

2. Le problème c'est que les clefs du dictionnaire sont peu lisibles. On peut néanmoins déviner quelques résultats.<br />
Quelle est la clef correspondant à la marque de l'appareil utilisé pour faire cette photo ? <br />
*Réponse : C'est la clef `271`.*<br />
Quelle est la clef correspondant aux coordonénes géographiques ? <br />
*Réponse : C'est la clef `34853`.*<br />
Que peut-on dire de la valeur associé à la clef `37500` ? <br />
*Réponse : La réponse est donnée en hexadécimale.*<br />

3. La clef `274` donne l'orientation de la photo suivant le schéma ci-dessous : <br/>
![](orientation.png) <br />
Quelle est l'orientation de la photo `chouchou0.jpg` ?

In [5]:
meta0[274]

1

*Réponse : 1, autrement dit, format paysage.*

4. Même chose pour la photo `chouchou1.jpg`.

In [6]:
# on procède comme au départ avec la première image :
# il faut charger l'image
photo1 = Image.open("chouchou1.jpg")
# on récupère les méta-données
meta1 = photo1._getexif()
# on interroge la bonne clef
meta1[274]

6

*Réponse : 6, autrement dit, format portrait.*

5. La clef `34853` donne les coordonnées géographiques de la prise de vue. <br />
Quelle est la structure de la valeur associée ?

In [7]:
meta0[34853]

{1: 'N', 2: (47.0, 27.0, 59.0), 3: 'W', 4: (0.0, 33.0, 5.0), 6: 0.0}

*Réponse : la valeur est elle-même un dictionnaire. <br />
La clef 1 donne l'orientation pour la latitude ('N' ou 'S'). <br />
La clef 2 donne la latitude en degré minute seconde sous la forme d'un tuple. <br/>
La clef 3 donne l'orientation pour la longitude ('E' ou 'W'). <br />
La clef 4 donne la longitude en degré minute seconde sous la forme d'un tuple.*

6. Donner les coordonnées géographiques de l'image `chouchou0.jpg` en valeurs décimales. Il faudra aussi tenir compte de l'orientation. <br />
(Par exemple pour la latitude, l'orientation est 'N' et si l'orientation est 'S' la latitude est négative)

In [8]:
def coord(meta) :
    # Récupération de la localisation terrestre
    gps = meta[34853]

    # gps est à nouveau un dictionnaire
    lat_dir = gps[1] #N ou S pour North ou South
    lat = gps[2]
    lon_dir = gps[3] #W ou E pour West ou East
    lon = gps[4]

    # Conversion décimale de la latitude et de la longitude
    latitude = lat[0] + lat[1]/60.+ lat[2]/3600.
    longitude = lon[0] + lon[1]/60. + lon[2]/3600.

    #On change le signe de la latitude si besoin
    if lat_dir == 'S':
        latitude = -1*latitude

    #On change le signe de la longitude si besoin
    if lon_dir == 'W':
        longitude = -1*longitude

    return latitude, longitude

In [9]:
coord(meta0)

(47.46638888888889, -0.5513888888888889)

## Insertion de photos sur une carte
On va utiliser le module `folium` qui permet de travailler sur des cartes d'OpenStreetMap.

Pour commencer on charge le module `folium` ...

In [10]:
import sys
!{sys.executable} -m pip install folium

... puis on l'importe.

In [11]:
# pour travailler sur des images numériques
import folium

Voici un petit programme pour comprendre comment on peut créer une carte.

In [12]:
def creer_carte(latitude,longitude):
    # création de la carte
    carte = folium.Map(
        # carte centrée sur le point de coordonnées (latitute, longitude)
        location = ([latitude , longitude]),
        #zoom de 16
        zoom_start =16)
    
    # création d'un marqueur pour les infos
    folium.Marker(
        # on positionne le marqueur
        [latitude , longitude],
        # commentaire sur le marqueur
        popup = "la photo a été prise ici",
        # type de marqueur
        icon = folium.Icon(color = 'red', icon = 'info−sign')
        # on rajoute ce marqueur à la carte
        ).add_to(carte)
    
    # on sauvegarde la carte
    carte.save('première_carte.html')

In [13]:
creer_carte(47.46638888888889, -0.5513888888888889)

Une autre version qui récupère les coordonnées directement dans les méta-données EXIF de la photo. <br />
De plus, l'icône sera la photo elle-même.

In [14]:
def creer_carte_photo(nom_photo):
    '''
    nom_photo est le nom de la photo sans son extension qui sera '.jpg'
    '''
    photo = Image.open(nom_photo + '.jpg')
    # récupération des données
    info_photo = photo._getexif()
    
    #récupération des coordonnées terrestres
    latitude, longitude = coord(info_photo)
    
    # création de la carte
    carte = folium.Map(
        # carte centrée sur le point de coordonnées (latitute, longitude)
        location = ([latitude , longitude]),
        #zoom de 16
        zoom_start =16)
    
    # création d'un marqueur pour les infos
    folium.Marker(
        # on positionne le marqueur
        [latitude , longitude],
        # commentaire sur le marqueur
        popup = "la photo a été prise ici",
        # création d’un îcone avec la photo
        icon = folium.CustomIcon(nom_photo+'.jpg', icon_size = (86,48))
        # on rajoute ce marqueur à la carte
        ).add_to(carte)
    
    # on sauvegarde la carte
    carte.save('carte_' + nom_photo + '.html')
    

In [15]:
creer_carte_photo('chouchou0')